##Install and imports

### install

In [ ]:
%%capture
%%bash
pip install -q -U bitsandbytes
pip install -q -U git+https://github.com/huggingface/transformers.git
pip install -q -U git+https://github.com/huggingface/peft.git
pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install jsonlines

### import

In [ ]:
import os
import json
import tqdm
import sys
import gzip
from collections import defaultdict

## Load model

In [ ]:
!pip install torch==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2

In [ ]:
%%capture
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True, padding=True, padding_side="left", maximum_length = 2048, model_max_length = 2048)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit = True, device_map = 'auto')
tokenizer.pad_token = tokenizer.eos_token
model.generation_config.pad_token_id = model.generation_config.eos_token_id

## Query expansion

### Load queries, candidate documents and qrels

In [ ]:
# Add needed imports and donwload queries and candidate documents files
!pip install sentence-transformers
import requests
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S')

!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-passagetest2019-top1000.tsv.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.2 MB/s eta 0:00:00
--2024-05-19 16:12:27--  https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
Resolving msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)... 20.150.34.1
Connecting to msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)|20.150.34.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4276 (4.2K) [application/x-gzip]
Saving to: ‘msmarco-test2019-queries.tsv.gz’

msmarco-test2019-qu 100%[===================>]   4.18K  --.-KB/s    in 0s      

2024-05-19 16:12:27 (1.25 GB/s) - ‘msmarco-test2019-queries.tsv.gz’ saved [4276/4276]

--2024-05-19 16:12:27--  https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-passagetest2019-top1000.tsv.gz
Resolving msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)... 20.150.34.1
Connecting to msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)|20

In [ ]:
# Download qrels file
def download_txt_file(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print("File downloaded successfully.")
    else:
        print("Failed to download file.")

# Example usage
url = "https://trec.nist.gov/data/deep/2019qrels-pass.txt"
save_path = "qrels.txt"
download_txt_file(url, save_path)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): trec.nist.gov:443
DEBUG:urllib3.connectionpool:https://trec.nist.gov:443 "GET /data/deep/2019qrels-pass.txt HTTP/1.1" 200 187092


File downloaded successfully.


In [ ]:
# Retrieve the queries
from sentence_transformers import util

queries = {}
queries_filepath = os.path.join('/content/msmarco-test2019-queries.tsv.gz')
if not os.path.exists(queries_filepath):
    logging.info("Download "+os.path.basename(queries_filepath))
    util.http_get('https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz', queries_filepath)

with gzip.open(queries_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

In [ ]:
#Read which passages are relevant
relevant_docs = defaultdict(lambda: defaultdict(int))
qrels_filepath = os.path.join("/content/qrels.txt")

if not os.path.exists(qrels_filepath):
    logging.info("Download "+os.path.basename(qrels_filepath))
    util.http_get('https://trec.nist.gov/data/deep/2019qrels-pass.txt', qrels_filepath)


with open(qrels_filepath) as fIn:
    for line in fIn:
        qid, _, pid, score = line.strip().split()
        score = int(score)
        if score > 0:
            relevant_docs[qid][pid] = score

# Only use queries that have at least one relevant passage
relevant_qid = []
for qid in queries:
    if len(relevant_docs[qid]) > 0:
        relevant_qid.append(qid)

In [ ]:
# Read the top 1000 passages that are supposed to be re-ranked
passage_filepath = os.path.join("/content/msmarco-passagetest2019-top1000.tsv.gz")

if not os.path.exists(passage_filepath):
    logging.info("Download "+os.path.basename(passage_filepath))
    util.http_get('https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-passagetest2019-top1000.tsv.gz', passage_filepath)



passage_cand = {}
with gzip.open(passage_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        qid, pid, query, passage = line.strip().split("\t")
        if qid not in passage_cand:
            passage_cand[qid] = []

        passage_cand[qid].append([pid, passage])

logging.info("Queries: {}".format(len(queries)))

INFO:root:Queries: 200


### Prtomp queries


In [ ]:
# Define the prompt template for query reformulation
prompt_template = "Answer the following query: {{query}} Give the rationale before answering"

In [ ]:
def reformulate_query_with_cot(original_query, model, tokenizer, max_length=512):
    # Using the prompt template from the scientific paper
    prompt_template = "Answer the following query: {query} Give the rationale before answering."
    prompt = prompt_template.format(query=original_query)

    # Encode and generate response
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

    # Decode the generated tokens to string
    new_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return new_query

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

# Generate reformulated queries using the updated function
reformulated_queries = {}
for qid in tqdm(relevant_qid, desc="Processing queries"):
    query = queries.get(qid, None)
    if query:
        reformulated_queries[qid] = reformulate_query_with_cot(query, model, tokenizer)

Processing queries:   0%|          | 0/43 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Processing queries: 100%|██████████| 43/43 [1:45:21<00:00, 147.00s/it]


In [ ]:
print(reformulated_queries)

{'156493': "Answer the following query: do goldfish grow Give the rationale before answering.\nanswer: Goldfish do grow, but at a very slow rate. The growth rate depends on various factors such as water temperature, quality of food, and the size of the fish. In ideal conditions, a juvenile golden ornamental fish can grow up to 1 inch (2.54 cm) per week during the first year of its life. After that, the rate of growth slows down significantly. However, it's essential to note that overfeeding and poor water quality can negatively impact the growth and health of fish, leading to stunted growth or other health issues.", '1110199': 'Answer the following query: what is wifi vs bluetooth Give the rationale before answering.\n answ 1: Bluetooth is a short-range wireless technology designed to replace cables and connect devices such as headphones, speakers, keyboards, mice, and smartphones. Wi-Fi, on the other hand, provides wireless high-speed internet connectivity over a longer range, allowin

In [ ]:
print(queries)

{'1108939': 'what slows down the flow of blood', '1112389': 'what is the county for grand rapids, mn', '792752': 'what is ruclip', '1119729': 'what do you do when you have a nosebleed from having your nose', '1105095': 'where is sugar lake lodge located', '1105103': 'where is steph currys home in nc', '1128373': 'iur definition', '1127622': 'meaning of heat capacity', '1124979': 'synonym for treatment', '885490': 'what party is paul ryan in', '1119827': 'cast of sky captain and the world of tomorrow', '190044': 'foods to detox liver naturally', '500575': "sop's policy", '883785': 'what origin is the last name goins', '264403': 'how long is recovery from a face lift and neck lift', '1108100': 'what type of movement do bacteria exhibit?', '421756': 'is prorate the same as daily rate', '1108307': 'what trail did thousands use to get to the gold rush', '966413': 'where are the benefits of cinnamon as a supplement?', '1111546': 'what is the medium for an artisan', '156493': 'do goldfish gro

### Evaluate cross-encoder
The comparison will be made on TinyBERT cross-encoder


In [ ]:
!pip install pytrec_eval

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308216 sha256=8b5a40e205c811fe5be2f409e99d43a5978e36393eabd44bb39a519380788b27
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


Evaluation without prompted queries

In [ ]:
import tqdm
import numpy as np
import pytrec_eval
from sentence_transformers.cross_encoder import CrossEncoder

# model_save_path = os.path.join("/content/MiniLM")
#model_save_path = os.path.join("/content/distilroBERTa")
model_save_path = os.path.join("/content/TinyBERT")


queries_result_list = []
run = {}
model = CrossEncoder(model_save_path, max_length=512)

for qid in tqdm.tqdm(relevant_qid):
    query = queries[qid]

    cand = passage_cand[qid]
    pids = [c[0] for c in cand]
    corpus_sentences = [c[1] for c in cand]

    cross_inp = [[query, sent] for sent in corpus_sentences]

    if model.config.num_labels > 1: #Cross-Encoder that predict more than 1 score, we use the last and apply softmax
        cross_scores = model.predict(cross_inp, apply_softmax=True)[:, 1].tolist()
    else:
        cross_scores = model.predict(cross_inp).tolist()

    cross_scores_sparse = {}
    for idx, pid in enumerate(pids):
        cross_scores_sparse[pid] = cross_scores[idx]

    sparse_scores = cross_scores_sparse
    run[qid] = {}
    for pid in sparse_scores:
        run[qid][pid] = float(sparse_scores[pid])

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda
  0%|          | 0/43 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

  2%|▏         | 1/43 [00:00<00:26,  1.57it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

  5%|▍         | 2/43 [00:01<00:23,  1.74it/s]

...

 95%|█████████▌| 41/43 [00:23<00:01,  1.57it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

 98%|█████████▊| 42/43 [00:23<00:00,  1.64it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 43/43 [00:24<00:00,  1.78it/s]


In [ ]:
evaluator = pytrec_eval.RelevanceEvaluator(relevant_docs, {'ndcg_cut.10', 'recall_100', 'map_cut.1000'})
scores = evaluator.evaluate(run)

print("Queries:", len(relevant_qid))
print("NDCG@10: {:.2f}".format(np.mean([ele["ndcg_cut_10"] for ele in scores.values()])*100))
print("Recall@100: {:.2f}".format(np.mean([ele["recall_100"] for ele in scores.values()])*100))
print("MAP@1000: {:.2f}".format(np.mean([ele["map_cut_1000"] for ele in scores.values()])*100))

Queries: 43
NDCG@10: 69.90
Recall@100: 50.47
MAP@1000: 45.55


Evaluation with prompted queries

In [ ]:
prompted_queries_result_list = []
prompted_run = {}
model = CrossEncoder(model_save_path, max_length=512)

for qid in tqdm.tqdm(relevant_qid):
    query = reformulated_queries[qid]

    cand = passage_cand[qid]
    pids = [c[0] for c in cand]
    corpus_sentences = [c[1] for c in cand]

    cross_inp = [[query, sent] for sent in corpus_sentences]

    if model.config.num_labels > 1: #Cross-Encoder that predict more than 1 score, we use the last and apply softmax
        cross_scores = model.predict(cross_inp, apply_softmax=True)[:, 1].tolist()
    else:
        cross_scores = model.predict(cross_inp).tolist()

    cross_scores_sparse = {}
    for idx, pid in enumerate(pids):
        cross_scores_sparse[pid] = cross_scores[idx]

    sparse_scores = cross_scores_sparse
    prompted_run[qid] = {}
    for pid in sparse_scores:
        prompted_run[qid][pid] = float(sparse_scores[pid])

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda
  0%|          | 0/43 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

  2%|▏         | 1/43 [00:01<00:58,  1.38s/it]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

  5%|▍         | 2/43 [00:02<00:57,  1.39s/it]

...

 95%|█████████▌| 41/43 [01:05<00:03,  1.68s/it]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

 98%|█████████▊| 42/43 [01:06<00:01,  1.71s/it]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 43/43 [01:08<00:00,  1.59s/it]


In [ ]:
evaluator_prompted = pytrec_eval.RelevanceEvaluator(relevant_docs, {'ndcg_cut.10', 'recall_100', 'map_cut.1000'})
scores_prompted = evaluator_prompted.evaluate(prompted_run)

print("TINYBER WITH PROMPTING")
print("Queries:", len(reformulated_queries))
print("NDCG@10: {:.2f}".format(np.mean([ele["ndcg_cut_10"] for ele in scores_prompted.values()])*100))
print("Recall@100: {:.2f}".format(np.mean([ele["recall_100"] for ele in scores_prompted.values()])*100))
print("MAP@1000: {:.2f}".format(np.mean([ele["map_cut_1000"] for ele in scores_prompted.values()])*100))

TINYBER WITH PROMPTING
Queries: 43
NDCG@10: 46.89
Recall@100: 40.73
MAP@1000: 32.94
